# Graphistry Netflow Demo

In this notebook, we will: 
- Set up [BlazingSQL](https://blazingsql.com) and the [RAPIDS AI](https://rapids.ai/) suite.
- Query 65M rows of network security data (netflow) with BlazingSQL and then pass to Graphistry to visualize and interact with the data.
![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-39814657-5&cid=555&t=event&ec=guides&ea=bsql-quick-start-guide&dt=bsql-quick-start-guide)

## Setup
### Environment Sanity Check 

RAPIDS packages (BlazingSQL included) require Pascal+ architecture to run. For Colab, this translates to a T4 GPU instance. 

The cell below will let you know what type of GPU you've been allocated, and how to proceed.

In [2]:
# tag specs
colab_smi = !nvidia-smi

# focus GPU type
try:
    my_gpu = ' '.join(colab_smi[7].split()[2:4])
# not on gpu acceleration 
except:
    raise Exception("\nPlease make sure you've configured Colab to request a GPU instance type.\n\n"
                    "At top of Colab, try: Runtime -> Change runtime type -> Hardware accelerator -> GPU -> Save\n")

# not allocated compatable GPU
if (my_gpu != b'Tesla T4') and (my_gpu != 'Tesla P100-PCIE...') and (my_gpu != 'GeForce GTX'):
    # allocated K80
    if my_gpu == 'Tesla K80':
        raise Exception("\nYou've been allocated a K80 instance\n\n"
                    "Unfortunately, this demo requires a T4 instance\n\n"
                    "At top of Colab, try: Runtime -> Reset all runtimes...\n")
    else:
        raise Exception(f"\nYou've achieved wizardy.\nyour GPU is {my_gpu}\nPlease inform info@blazingsql.com")

# allocated compatable GPU
else:
    print('Woo! You got the right kind of GPU!')

Woo! You got the right kind of GPU!


## Installs 

Below you will find three code blocks:
1. The first installs miniconda.
2. The second installs RAPIDS AI and sets up the system environment. 
3. The third installs BlazingSQL.

### Miniconda

In [3]:
# intall miniconda
!wget -c https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local

### RAPIDS AI

In [4]:
# install RAPIDS packages
!conda install -q -y --prefix /usr/local -c nvidia -c rapidsai \
  -c numba -c conda-forge -c pytorch -c defaults \
  cudf=0.9 cuml=0.9 cugraph=0.9 python=3.6 cudatoolkit=10.0

# set environment vars
import sys, os, shutil
sys.path.append('/usr/local/lib/python3.6/site-packages/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'

# copy .so files to current working dir
for fn in ['libcudf.so', 'librmm.so']:
    shutil.copy('/usr/local/lib/'+fn, os.getcwd())

### BlazingSQL

In [ ]:
# Install BlazingSQL for CUDA 10.0
! conda install -q -y --prefix /usr/local -c conda-forge -c defaults -c nvidia -c rapidsai \
   -c blazingsql/label/cuda10.0 -c blazingsql \
   blazingsql-calcite blazingsql-orchestrator blazingsql-ral blazingsql-python

!pip install flatbuffers

# Download CSV

You will need to download the CSV we are going to use for this demo.

In [ ]:
!wget https://blazingsql-colab.s3.amazonaws.com/netflow_data/nf-chunk2.csv

--2019-08-23 21:43:50--  https://blazingsql-colab.s3.amazonaws.com/netflow_data/nf-chunk2.csv
Resolving blazingsql-colab.s3.amazonaws.com (blazingsql-colab.s3.amazonaws.com)... 52.216.137.76
Connecting to blazingsql-colab.s3.amazonaws.com (blazingsql-colab.s3.amazonaws.com)|52.216.137.76|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2725056295 (2.5G) [text/csv]
Saving to: ‘nf-chunk2.csv’

nf-chunk2.csv       100%[===================>]   2.54G  49.2MB/s    in 56s     

2019-08-23 21:44:46 (46.2 MB/s) - ‘nf-chunk2.csv’ saved [2725056295/2725056295]



#Load & Query Tables

Here we are importing cuDF and BlazingSQL. We are then loading the CSV files into a GPU DataFrame (gdf), and then creating tables so that we can run SQL queries on those GDFs. 

Note, when you create a table off of a GDF there is no copy, it is merely registering the schema.

In [ ]:
# Set Environment Variables

import sys, os

sys.path.append('/usr/local/lib/python3.6/site-packages/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'

#Standup the BlazingSQL Services - We are working on removing the need to call these functions and just initializing them in BlazingContext
import subprocess
subprocess.Popen(['blazingsql-orchestrator', '9100', '8889', '127.0.0.1', '8890'],stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
subprocess.Popen(['java', '-jar', '/usr/local/lib/blazingsql-algebra.jar', '-p', '8890'])
import pyblazing.apiv2.context as cont
cont.runRal()
time.sleep(1) #Wait for service to start.

## Import packages and create Blazing Context
You can think of the BlazingContext much like a Spark Context (i.e. where information such as FileSystems you have registered and Tables you have created will be stored). If you have issues running this cell, restart runtime and try running it again.


In [ ]:
from blazingsql import BlazingContext
import cudf

bc = BlazingContext()

connection established
CPU times: user 1.91 ms, sys: 13.2 ms, total: 15.1 ms
Wall time: 38 ms


In [ ]:
#Load CSVs into GPU DataFrames (gdf)
netflow_gdf = cudf.read_csv('/content/nf-chunk2.csv')

#Create BlazingSQL Tables - There is no copy in this process
bc.create_table('netflow', netflow_gdf)

From there we can simply run a SQL query.

In this example we are taking millions of rows of netflow (network flow) data in order to search for anomalous activity within a network.

We are going to run some joins and aggregations in order to condese these millions of rows into thousands of rows that represent nodes and edges.

In [ ]:
%%time
sql = '''
SELECT
  a.firstSeenSrcIp as source,
  a.firstSeenDestIp as destination,
  count(a.firstSeenDestPort) as targetPorts,
  SUM(a.firstSeenSrcTotalBytes) as bytesOut,
  SUM(a.firstSeenDestTotalBytes) as bytesIn,
  SUM(a.durationSeconds) as durationSeconds,
  MIN(parsedDate) as firstFlowDate,
  MAX(parsedDate) as lastFlowDate,
  COUNT(*) as attemptCount
  FROM
  main.netflow a
  GROUP BY
  a.firstSeenSrcIp,
  a.firstSeenDestIp
  '''

result = bc.sql(sql).get()
result_gdf = result.columns
df = result_gdf.to_pandas()
print(df.head(10))

  firstSeenSrcIp firstSeenDestIp  ...         lastFlowDate  attemptCount
0    172.30.2.60        10.0.0.9  ...  2013-04-03 12:12:37            82
1   172.10.1.162       10.0.0.11  ...  2013-04-03 14:58:35            87
2   172.10.1.234        10.0.0.5  ...  2013-04-03 15:11:07           104
3      10.1.0.76     172.10.1.82  ...  2013-04-03 09:55:05             1
4    172.10.1.89        10.0.0.5  ...  2013-04-03 15:17:39           112
5   172.30.1.201       172.0.0.1  ...  2013-04-03 23:06:00            29
6   172.10.1.106    10.199.250.2  ...  2013-04-03 10:12:35            40
7    172.30.1.10       10.0.0.12  ...  2013-04-03 12:11:40            69
8    172.20.1.58        10.7.5.5  ...  2013-04-03 11:20:09            49
9   172.30.2.125        10.0.0.9  ...  2013-04-03 12:12:37            69

[10 rows x 9 columns]
CPU times: user 59.7 ms, sys: 10.1 ms, total: 69.7 ms
Wall time: 2.28 s
